<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [135]:
import numpy as np
import pandas as pd
import seaborn as sns
import random as rn

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

А запилим-ка мы игрушечный датасет в котором стоимость б/у машин будет линейно зависеть от возраста, прожорливости двигателя и транспортного налога на тачку, при этом в качестве базовых возьмем разные параметры движка и прочую мишуру

In [80]:
random.seed(42)
n_samples = 1000

Я, конечно, немного угорел, включив в список 16ти цилиндровых монстров, тем не менее, такие машины существуют и существовали годов с 1930-х, хоть их и было мало, объем двигателя прикинут довольно реалистично, мощность тоже, за исключением некоторого провала на максимальном объеме, но я уж не стал настолько сложные модели строить.

In [81]:
# сгенерируем год
my_list = [i for i in range(1975, 2021, 1)]   
year_list = [rn.choice(my_list) for i in range(0, n_samples)]

# число цилиндров
my_list = [4,6,8,10,12,16]
engine_list = [rn.choice(my_list) for i in range(0, n_samples)]

# прожорливость
fuel_consumption_list = [round((engine_list[i] * 1.2 + rn.uniform(-1, 5)),2) for i in range(0, n_samples)]   

# объем двигателя 
displacement_list = [int((engine_list[i] * 0.5 + rn.uniform(-1, 4))*1000) for i in range(0, n_samples)]

# мощность
power_list = [int(((engine_list[i]**2)*2.6 + rn.uniform(-20, 30))) for i in range(0, n_samples)]

In [82]:
# скрытый параметр - русский транспортный налог на 2021, 
# посчитал его в баксах для удобства
def foo(arg):
    buck = 72.25
    if arg < 100:
        return (arg * 2.5)/buck
    elif (arg >= 100) and (arg <= 150):
        return (arg * 3.5)/buck
    elif (arg > 150) and (arg <= 200):
        return (arg * 5)/buck
    elif (arg > 200) and (arg <= 250):
        return (arg * 7)/buck
    elif arg > 250:
        return (arg * 15)/buck

taxe_list = [round((foo(power_list[i])), 2) for i in range(0, n_samples)]

# скрытый параметр - сколько лет машине
years_old_list = [2021 - year_list[i]  for i in range(0, n_samples)]

In [129]:
price_list = [int(taxe_list[i] * 2000 - fuel_consumption_list[i] * 500 - years_old_list[i] * 741 + 35000) for i in range(0, n_samples)]

In [130]:
min(price_list)

69

In [131]:
max(price_list)

311234

в целом довольно адекватно - от 311к баксов за современный суперкар до 70 баксов за развалюху 40 летней древности

In [134]:
data = pd.DataFrame({'Year': year_list, 'Engine': engine_list, 'Fuel_consumption': fuel_consumption_list, 'Displacement': displacement_list, 'Power':power_list, 'Price':price_list})
data.head(20)

,Year,Engine,Fuel_consumption,Displacement,Power,Price
0,2015,4,8.07,2715,22,28039
1,1982,16,21.30,7546,670,273651
2,1976,6,6.95,6000,79,3640
3,1992,8,9.10,7649,181,34021
4,1990,16,19.16,10702,685,286869
5,1989,12,16.10,7579,373,158118
6,1983,16,21.28,9759,677,277302
7,1981,6,11.19,3654,112,10625
8,2018,10,11.55,6131,271,139522
9,2009,4,9.26,3310,51,24998


вообще довольно прикольно получилось, так можно, например, генерировать стоимость лута в игрушках

ну и пошли угорать

In [136]:
X = data[['Year', 'Engine', 'Fuel_consumption', 'Displacement', 'Power']]
y = data['Price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Year', 'Engine', 'Fuel_consumption', 'Displacement', 'Power']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 7.59231831e+02 -3.19728950e+03 -5.96284796e+02  3.49805322e-01
  5.27633414e+02]
Bias: -1518712.4833129747
Error: 13983.459048017727


а теперь учтем налог и возраст

In [140]:
data['Years_old'] = pd.DataFrame({'Years_old': years_old_list}) 
data['Taxe'] = pd.DataFrame({'Taxe': taxe_list})
data.head()

,Year,Engine,Fuel_consumption,Displacement,Power,Price,Years_old,Taxe
0,2015,4,8.07,2715,22,28039,6,0.76
1,1982,16,21.30,7546,670,273651,39,139.10
2,1976,6,6.95,6000,79,3640,45,2.73
3,1992,8,9.10,7649,181,34021,29,12.53
4,1990,16,19.16,10702,685,286869,31,142.21


In [142]:
X = data[['Years_old', 'Engine', 'Fuel_consumption', 'Displacement', 'Power', 'Taxe']]
y = data['Price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Years_old', 'Engine', 'Fuel_consumption', 'Displacement', 'Power', 'Taxe']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-7.41000000e+02 -5.86197757e-12 -5.00000000e+02  4.07140988e-12
  3.19300142e-13  2.00000000e+03]
Bias: 34999.99999997423
Error: 8.212267857743428e-09


**В разы улучшилось, даже с учетом что мы не снесли лишние параметры**